In [162]:
import pandas as pd

#### Data sample: _Banca Transilvania transaction report_

In [ ]:
# read rows 9-12
df_metadata = pd.read_csv(r".csv", skiprows=9, nrows=4)
df_metadata.columns = [0, 1]
df_metadata
# extract account currency
account_currency = df_metadata.iloc[1, 1].split(" ")[-1]
account_currency

In [ ]:
df = pd.read_csv(r".csv", skiprows=14)
df.head()

#### Data processing

##### Actual transaction date extraction

In [165]:
# Extract date from "Description" column; format is dd/mm/yyyy
df["Date"] = df["Description"].str.extract(r"(\d{2}/\d{2}/\d{4})")

In [ ]:
mask_isna_actual_transaction_date = df["Date"].isna()
mask_isna_actual_transaction_date.value_counts()

In [ ]:
df[mask_isna_actual_transaction_date].head()

In [ ]:
df.loc[mask_isna_actual_transaction_date, "Date"] = df.loc[mask_isna_actual_transaction_date, "Processing date"]
mask_isna_actual_transaction_date = df["Date"].isna()
mask_isna_actual_transaction_date.value_counts()

In [ ]:
df.head()

##### Transaction type feature engineering

In [170]:
df["Transaction type"] = df.apply(lambda row: "Income" if pd.notna(row["Credit"]) else ("Expense" if pd.notna(row["Debit"]) else "Unknown"), axis=1)

In [ ]:
df.head()

In [ ]:
df["Transaction type"].value_counts()

##### Extract currency

In [ ]:
# extract currency from "Description" column
df["Currency"] = df["Description"].str.extract(r"(RON|EUR|USD)")
df["Currency"].value_counts()

In [ ]:
df["Currency"].isna().value_counts()

In [175]:
df = df.fillna({"Currency": account_currency})

In [ ]:
df.head()

##### Description relevant text extraction

In [ ]:
df["Description"].head().apply(lambda x: x.split(";")[1]).tolist()

In [178]:
df["Cleaned Transaction Description"] = df["Description"].apply(lambda x: x.split(";")[1])
# erase substrings with pattern "TID:XXXXXXXX"
df["Cleaned Transaction Description"] = df["Cleaned Transaction Description"].str.replace(r"TID:\w+", "", regex=True)
# erase substrings with pattern "comision tranzactie XX.XX RON"
df["Cleaned Transaction Description"] = df["Cleaned Transaction Description"].str.replace(r"comision tranzactie \d+\.\d+ RON", "", regex=True)
# erase substrings with pattern "RRN:XXXXXXXX"
df["Cleaned Transaction Description"] = df["Cleaned Transaction Description"].str.replace(r"RRN:\w+", "", regex=True)
# erase substrings with pattern "dd/mm/yyyy XXXXXXXX"
df["Cleaned Transaction Description"] = df["Cleaned Transaction Description"].str.replace(r"\d{2}/\d{2}/\d{4} \w+", "", regex=True)
# replace multiple spaces with single space
df["Cleaned Transaction Description"] = df["Cleaned Transaction Description"].str.replace(r"\s+", " ", regex=True)
# remove leading and trailing spaces
df["Cleaned Transaction Description"] = df["Cleaned Transaction Description"].str.strip()
# erase substrings with pattern "RRN: XXXXXXXX"
df["Cleaned Transaction Description"] = df["Cleaned Transaction Description"].str.replace(r"RRN: \w+", "", regex=True)

In [ ]:
df["Cleaned Transaction Description"].head().tolist()

In [ ]:
df.head()

##### Keep amount in dedicated column

In [ ]:
# extract amount from "Debit" or "Credit" column, whichever is populated
df["Amount"] = df.apply(lambda row: row["Debit"] if pd.notna(row["Debit"]) else row["Credit"], axis=1)
df.head()

In [ ]:
columns = ["Transaction type", "Category", "Label", "Date", "Cleaned Transaction Description", "Notes", "Amount", "Currency"]
df_processed = pd.DataFrame(columns=columns)
# copy all columns from df to df_processed if column name matches
for column in df.columns:
    if column in columns:
        df_processed[column] = df[column].copy()
df_processed.head()